# <center> STATS 607 - LECTURE 6
## <center> 09/24/2018

# <center> Python Internals

Thank you Prof. Kerby Shedden for making available this material.

Scripts written in the Python language are usually interpreted, and several interpreters have been written that are capable of executing most valid Python scripts. The vast majority of Python users use the Python interpreter implemented in C that is called “CPython” (this is the interpreter available from www.python.org).

The CPython interpreter is a very mature and complex piece of C software. Most Python programmers do not need to understand much about the internal implementation and design of CPython. However there are a few things that are worth being aware of, for example the implementation details of the core data structures, and the implementation of the garbage collectors. We will discuss these topics here.

## Lists

Python lists are actually arrays, not linked lists. This means that a value can be selected from the list in a constant amount of time, rather than in an amount of time that grows linearly with the length of the list. The downside of using arrays for storing sequential data is that they cannot be enlarged – when the storage is used up it is necessary to allocate new storage and copy the entire list to the new location, freeing the current memory chunk to be garbage collected. To prevent excessive copying, when the array is reallocated around 12.5% more space than is needed is reserved, so there is some room to grow before reallocating again.

Since lists in Python store heterogeneous data, all the list contents are stored via indirection (i.e. by pointers). This means that the contiguous memory backing a list a consists of len(a) pointers, occupying a total of 8*len(a) bytes. Each pointer in turn points to the actual data value. So when you retrieve the value a[10], the interpreter first looks up the pointer back(a)[10] in the backing array of a, then returns the object located at this address.

Lets look at the following code blocks:

In [20]:
import sys

print(sys.getsizeof(1)) # Returns the size 

28


In [14]:
print(sys.getsizeof([]))
print(sys.getsizeof([1]))
print(sys.getsizeof([1,2]))
print(sys.getsizeof([1,2,3]))

64
72
80
88


In [15]:
x=[]
for i in range(20):
    x.append(10)
    print(sys.getsizeof(x),end=' ')    

96 96 96 96 128 128 128 128 192 192 192 192 192 192 192 192 264 264 264 264 

In [16]:
x = []
s = sys.getsizeof(x)
r = []
for i in range(100000):
    x.append(0)
    t = sys.getsizeof(x)
    if t != s:
        d = t - s
        r.append((len(x) - 1, len(x), s, t, d, round(d / s,3), d / 8))
        s = t

In [17]:
r

[(0, 1, 64, 96, 32, 0.5, 4.0),
 (4, 5, 96, 128, 32, 0.333, 4.0),
 (8, 9, 128, 192, 64, 0.5, 8.0),
 (16, 17, 192, 264, 72, 0.375, 9.0),
 (25, 26, 264, 344, 80, 0.303, 10.0),
 (35, 36, 344, 432, 88, 0.256, 11.0),
 (46, 47, 432, 528, 96, 0.222, 12.0),
 (58, 59, 528, 640, 112, 0.212, 14.0),
 (72, 73, 640, 768, 128, 0.2, 16.0),
 (88, 89, 768, 912, 144, 0.188, 18.0),
 (106, 107, 912, 1072, 160, 0.175, 20.0),
 (126, 127, 1072, 1248, 176, 0.164, 22.0),
 (148, 149, 1248, 1448, 200, 0.16, 25.0),
 (173, 174, 1448, 1672, 224, 0.155, 28.0),
 (201, 202, 1672, 1928, 256, 0.153, 32.0),
 (233, 234, 1928, 2216, 288, 0.149, 36.0),
 (269, 270, 2216, 2536, 320, 0.144, 40.0),
 (309, 310, 2536, 2896, 360, 0.142, 45.0),
 (354, 355, 2896, 3304, 408, 0.141, 51.0),
 (405, 406, 3304, 3760, 456, 0.138, 57.0),
 (462, 463, 3760, 4272, 512, 0.136, 64.0),
 (526, 527, 4272, 4848, 576, 0.135, 72.0),
 (598, 599, 4848, 5496, 648, 0.134, 81.0),
 (679, 680, 5496, 6232, 736, 0.134, 92.0),
 (771, 772, 6232, 7056, 824, 0.132, 

Note also that the memory size of the list itself is roughly 8 times the length of the list, regardless of what is stored in the list:

In [18]:
x = [1 for k in range(10000)]
y = [1 for k in range(100000)]
z = [list(x) for k in range(10000)]

print(sys.getsizeof(x))
print(sys.getsizeof(y))
print(sys.getsizeof(z))

87624
824464
87624


The overhead of growing a list is not great compared to the overhead of accessing elements from an existing list:

In [19]:
m = 10000
x = [i for i in range(m)]

def f():
    y = []
    for i in range(m):
        y.append(i)

def g():
    z = 0
    for i in x:
        z += i

%timeit f()
%timeit g()

938 µs ± 6.05 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
514 µs ± 16.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Dictionaries

Python dictionaries, like Python lists, are actually arrays. But since the keys of a dictionary are not necessarily integers, there needs to be a fast way to determine the array position corresponding to a key. This is done using hash functions. They goal of a hash function is to map an arbitrary data object to an integer value in a deterministic way. So, the ideal hash should use the entire range of possible hash values, and as much as possible, should distribute most input sequences approximately uniformly within its range.

The Python hash function is built-in and available as the function hash, e.g.:

In [1]:
print(hash(13349384082))
print(hash("dfjoafiojf"))
print(hash((1, "cat", (3, 45.6))))

13349384082
-126669421187381922
-2143251727911499404


Note that hash can be called on many but not all compound data objects, e.g. you cannot call hash on a list.

The Python dictionary implementation is actually represented by an array of indices ('indexTable') and a table ('Table'). To illustrate, suppose we have the current state for our dictionary:

In [2]:
d = {"keys": 10, "in": 20, "a": 30, "dictionary": 40}

'indexTable' would look like the output of the following code:

In [3]:
indexTable = [None for i in range(8)]
keys = list(d.keys())
for i in range(len(keys)):
    indexTable[hash(keys[i]) % 8] = i
indexTable

[2, None, None, None, None, 1, 0, 3]

'table' would look like the output of the following code:

In [4]:
table = []
for i in range(len(d)):
    table.append((hash(keys[i]), keys[i], d[keys[i]]))
table

[(6004549795674874742, 'keys', 10),
 (1304138452349696829, 'in', 20),
 (8885219270625913808, 'a', 30),
 (-1182809854426603257, 'dictionary', 40)]

If we want to retrieve the value corresponding to key “in”, we call hash("in") % 8 (since the arrays currently have length 8).  This corresponds to the index in 'indexTable' wich value corresponds to a position in 'Table' where the relevant 'Value' is expected to be.

In [5]:
search='in'
candidateTuple = table[indexTable[hash(search) % 8]]
candidateTuple

(1304138452349696829, 'in', 20)

In [6]:
if (candidateTuple[0] == hash(search)):
    value = candidateTuple[2]
value

20

Note: For reasons we won’t get into here, hash values are not stable between sessions.

Now suppose we want to put the key:value pair "another key":50 into the dictionary. We find that hash("another key") % 8 is 5, but slot 5 in 'indexTable' is already assigned to a different key (“keys”). This is called a “hash collision”.

In [7]:
pos = hash('another key') % 8
pos

5

In [8]:
print(indexTable)
table[indexTable[pos]]

[2, None, None, None, None, 1, 0, 3]


(1304138452349696829, 'in', 20)

To resolve this collision, we apply the transformation 5j + 1 % 8 to the index. Starting with j = 2 we get 3, which is an empty slot.

In [9]:
j = 2
while indexTable[pos]!=None:
    print('The slot', pos, 'in indexTable is occupied!')
    pos = (5*j + 1) % 8
    print('We will now try: ', pos)
    j+=1
    
print('\nWe will use slot:', pos)

The slot 5 in indexTable is occupied!
We will now try:  3

We will use slot: 3


The state of the 'indexTable' and 'table' will now be:

In [10]:
indexTable[pos] = len(table)
indexTable

[2, None, None, 4, None, 1, 0, 3]

In [11]:
table.append((hash('another key'), 'another key', 50))
table

[(6004549795674874742, 'keys', 10),
 (1304138452349696829, 'in', 20),
 (8885219270625913808, 'a', 30),
 (-1182809854426603257, 'dictionary', 40),
 (-1406979654865585939, 'another key', 50)]

Note that the collision resolution also has to be considered during lookups. If we want to retrieve the value for “another key”, we would first look in position 5, then when we see that the keys and the hash values do not match hash("another key"), we continue to the next slot given by 5*j + 1 % 8 until we find the matching hash.

In [12]:
print(table[indexTable[hash('another key') % 8]])
print(table[indexTable[(5*2 + 1) % 8]])

(1304138452349696829, 'in', 20)
(-1406979654865585939, 'another key', 50)


Once the backing arrays for a dictionary are around two thirds full, the length of the backing arrays will be increased by a factor of 2, and the old arrays will be transferred into the new space.

The backing arrays always have length equal to a power of 2. A basic fact of number theory is that the recursion j : 5*j + 1 mod 2^p will visit each integer between 0 and 2^p - 1 once before repeating, so we will never fail to find an empty slot.

Now we will do some profiling. First we compare the timing of dictionary creation versus array creation:

In [21]:
m = 10000

def f():
    x = [i for i in range(m)]

def g():
    x = {i: i for i in range(m)}

%timeit f()
%timeit g()

455 µs ± 57.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
648 µs ± 10.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Here I get that the dictionary creation is slower by around a factor of 1.6. Now, we can compare the timing of array lookups and dictionary lookups:

In [22]:
m = 10000
x = [i for i in range(m)]
y = {i : i for i in range(m)}

def f():
    for i in range(m):
        x[i] += 1

def g():
    for i in range(m):
        y[i] += 1

%timeit f()
%timeit g()

1.28 ms ± 119 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.65 ms ± 19.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


I get that the dictionary lookups are slower by around a factor of 1.4, which is not bad given the extra steps that are needed for dictionary lookups.

## Garbage Collection

A Python variable is a symbol that refers to a value. The symbol exists only in the source code and in some internal data structures managed by the interpreter. The value is stored in memory. Since Python allows easy creation of references, the same memory can be referred to by multiple variables. To check whether two variables are references to the same value, use the built-in 'id' function:

In [23]:
x = [1, 2, 3]
y = x
print(id(x))
print(id(y))

4371871368
4371871368


Python uses a technique called reference counting to keep track of how many variables refer to each object. We can obtain the reference count using the sys.getrefcount function:

In [24]:
x = [1, 2, 3]
y = x
z = [1, x]
print(sys.getrefcount(x))

4


Note that the reference count will always be 1 greater than you might expect, because a temporary variable bound to the value of x is called when you make the call to sys.getrefcount(x).

You can use the del function to delete variables and check how the reference count is adjusted:

In [25]:
x = [1, 2, 3]
y = x
print(sys.getrefcount(x))
del y
print(sys.getrefcount(x))

3
2


When the reference count of an object becomes zero, it can be garbage collected. This does not mean however that it will immediately be garbage collected. The garbage collector runs on a low priority thread, so if there is plenty of free memory, the garbage collector may not act immediately.

There is one important complexity that reference counting does not handle, which is cycles of references. For example, it is possible to nest a list inside itself, like this:


In [26]:
x = [1, 2]
x.append(x)

Now try the following:

In [27]:
print(x[2])
print(x[2][2])
print(x[2][2][2])

[1, 2, [...]]
[1, 2, [...]]
[1, 2, [...]]


Clearly we could go on like this forever. Since x refers to itself, its reference count should be infinite. But due to some bounds imposed by the interpreter, it will actually not be:

In [28]:
print(sys.getrefcount(x))

3


The challenge for the garbage collector is that if a variable like this goes out of scope, its reference count may not drop to zero. Thus, the reference counting garbage collector may never collect it, and we could have a memory leak. To address this, Python has a second garbage collector that uses a different strategy called mark and sweep. This garbage collector is able to detect and remove cycles, but it is more expensive to run. In practice, if there are no circular references in your code, most or all garbage collection will be handled by the simpler reference counting garbage collector, and the mark and sweep collector will not have much to do.